# Engineer the features


<a name="sumario"></a>
# Sumário
<!--ts-->
   * [Importe das Bibliotecas](#import)
   * [Leitura dos dados](#leitura)
   * [Escalar o AGE_PERCENTIL](#age)
   * [Drop WINDOW](#corr)
<!--te-->

<a name="import"></a>
# Importe das Bibliotecas

## Bibliotecas Externas

In [77]:
import pandas as pd
import numpy as np

from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from sklearn.ensemble import RandomForestClassifier

## Bibliotecas Internas

In [2]:
import sys
sys.path.insert(1, "../src/")
from features import scale_age_percentil

-----------------
Retornar ao [Sumário](#sumario)

<a name="leitura"></a>
# Leitura dos dados
Faz a leitura dos dados que serão usados para a limpeza e futura modelagem.<br>
**```df : pd.DataFrame```** é o DataFrame que receberá os valores *raw* baixados do Kaggle.

In [3]:
# Leitura do raw data desse  projeto no Github
df = pd.read_csv('../data/interim/df_7_cleaned.csv', index_col='PATIENT_VISIT_IDENTIFIER')

# Imprime na tela todas as colunas em vez de as 10 primeiras e 10 ultimas
pd.set_option('max_columns', df.shape[1])
df.head()

,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,ALBUMIN,BE_ARTERIAL,BE_VENOUS,BIC_ARTERIAL,BIC_VENOUS,BILLIRUBIN,BLAST,CALCIUM,CREATININ,FFA,GGT,GLUCOSE,HEMATOCRITE,HEMOGLOBIN,INR,LACTATE,LEUKOCYTES,LINFOCITOS,NEUTROPHILES,P02_ARTERIAL,P02_VENOUS,PC02_ARTERIAL,PC02_VENOUS,PCR,PH_ARTERIAL,PH_VENOUS,PLATELETS,POTASSIUM,SAT02_ARTERIAL,SAT02_VENOUS,SODIUM,TGO,TGP,TTPA,UREA,DIMER,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_MEDIAN,BLOODPRESSURE_SISTOLIC_MEDIAN,HEART_RATE_MEDIAN,RESPIRATORY_RATE_MEDIAN,TEMPERATURE_MEDIAN,OXYGEN_SATURATION_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,OXYGEN_SATURATION_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,BLOODPRESSURE_DIASTOLIC_DIFF,BLOODPRESSURE_SISTOLIC_DIFF,HEART_RATE_DIFF,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,WINDOW,ICU
PATIENT_VISIT_IDENTIFIER,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,-1.0,-1.0,-0.317073,-0.317073,-0.938950,-1.0,0.183673,-0.868365,-0.742004,-0.945093,-0.891993,0.090147,0.109756,-0.932246,1.000000,-0.835844,-0.914938,-0.868747,-0.170732,-0.704142,-0.77931,-0.754601,-0.875236,0.234043,0.363636,-0.540721,-0.518519,0.939394,0.345679,-0.028571,-0.997201,-0.990854,-0.825613,-0.836145,-0.994912,0.086420,-0.230769,-0.283019,-0.593220,-0.285714,0.736842,0.086420,-0.230769,-0.283019,-0.586207,-0.285714,0.736842,0.237113,0.0000,-0.162393,-0.500000,0.208791,0.898990,-0.247863,-0.459459,-0.432836,-0.636364,-0.420290,0.736842,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,1
2,0,10th,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.605263,-1.0,-1.0,-0.317073,-0.317073,-0.938950,-1.0,0.357143,-0.912243,-0.742004,-0.958528,-0.780261,0.144654,0.158537,-0.959849,1.000000,-0.382773,-0.908714,-0.412965,-0.170732,-0.704142,-0.77931,-0.754601,-0.939887,0.234043,0.363636,-0.399199,-0.703704,0.939394,0.345679,0.085714,-0.995428,-0.986662,-0.846633,-0.836145,-0.978029,-0.489712,-0.685470,-0.048218,-0.645951,0.357143,0.935673,-0.506173,-0.815385,-0.056604,-0.517241,0.357143,0.947368,-0.525773,-0.5125,-0.111111,-0.714286,0.604396,0.959596,-0.435897,-0.491892,0.000000,-0.575758,0.101449,1.000000,-0.547826,-0.533742,-0.603053,-0.764706,-1.000000,-0.959596,-0.515528,-0.351328,-0.747001,-0.756272,-1.000000,-0.961262,0-2,1
3,0,40th,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.263158,-1.0,-1.0,-0.317073,-0.317073,-0.972789,-1.0,0.326531,-0.968861,-0.194030,-0.316589,-0.891993,-0.203354,-0.219512,-0.959849,-0.828421,-0.729239,-0.836100,-0.784714,-0.170732,-0.633136,-0.77931,-0.779141,-0.503592,0.234043,0.363636,-0.564753,-0.777778,0.939394,0.580247,0.200000,-0.989549,-0.956555,-0.846633,-0.937349,-0.978029,0.012346,-0.369231,-0.528302,-0.457627,-0.285714,0.684211,0.012346,-0.369231,-0.528302,-0.448276,-0.285714,0.684211,0.175258,-0.1125,-0.384615,-0.357143,0.208791,0.878788,-0.299145,-0.556757,-0.626866,-0.515152,-0.420290,0.684211,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
4,0,10th,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.605263,-1.0,-1.0,-0.317073,-0.317073,-0.935113,-1.0,0.357143,-0.913659,-0.829424,-0.938084,-0.851024,0.358491,0.304878,-0.959849,1.000000,-0.702202,-0.641079,-0.812725,-0.170732,-0.704142,-0.77931,-0.754601,-0.990926,0.234043,0.363636,-0.457944,-0.592593,0.939394,0.345679,0.

<a name="age"></a>
# Escalar o AGE_PERCENTIL
A *feature* chamada ```AGE_PERCENTIL``` é uma *feature* categórica. A categoria funciona da seguinte maneira:<br>
Inicialmente os dados são ordenados por idade, do mais novo para o mais velho, e em seguida separados nos percentis, ex:
10th é a categoria que representa os 10% de pacientes com a menor idade, já o 20th representa o intervalo de pacientes que estão entre os 10% e 20% da idade e assim por diante [veja aqui](https://www.kaggle.com/S%C3%ADrio-Libanes/covid19/discussion/276074).<br>
Como alguns modelos não trabalham com dados categóricos, seria interessante, caso possível, transformar esse dado categórico em um valor real entre ma determinada faixa.<br>
Como alguns modelos consideram a magnitude dos valores como peso para a feature, não é razoavel ter todos os dados do dataset entre 0 - 1 ou (-1) - 1, e uma única feature estar de 0 - 100 por exemplo. Por isso iremos escalar o ```AGE_PERCENTIL``` entre 0 - 1.<br>
**```df_1_agescaled```** vai receber a resposta da função ```scale_age_percentil``` com os dados normalizados entre os valores 0 -  1. 

In [4]:
df['AGE_PERCENTIL'].value_counts()

20th          42
30th          39
40th          38
10th          36
80th          36
70th          34
50th          34
Above 90th    34
60th          30
90th          28
Name: AGE_PERCENTIL, dtype: int64

In [5]:
df_1_agescaled = scale_age_percentil(df)

In [6]:
df_1_agescaled['AGE_SCALED'].value_counts()

0.111111    42
0.222222    39
0.333333    38
0.000000    36
0.777778    36
0.666667    34
0.444444    34
1.000000    34
0.555556    30
0.888889    28
Name: AGE_SCALED, dtype: int64

-----------------
Retornar ao [Sumário](#sumario)


# Drop WINDOW feature


In [8]:
df_2_wo_window = df_1_agescaled.drop('WINDOW', axis=1)

-----------------
Retornar ao [Sumário](#sumario)

# Remove 'quasi constant features'

https://towardsdatascience.com/how-to-use-variance-thresholding-for-robust-feature-selection-a4503f2b5c3f
https://machinelearningmastery.com/discover-feature-engineering-how-to-engineer-features-and-how-to-get-good-at-it/
https://www.youtube.com/watch?v=JEfBvK91CXc&t=329s


In [70]:
y = df_2_wo_window["ICU"]
X = df_2_wo_window.drop(["ICU"], axis=1)


In [145]:
vt = VarianceThreshold(0.05)
_ = vt.fit(X, y)

mask = vt.get_support()

X_1 = X.loc[:, mask]
X_1

,AGE_ABOVE65,AGE_SCALED,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 3,DISEASE GROUPING 5,HTN,IMMUNOCOMPROMISED,OTHER,HEMOGLOBIN,LACTATE,PCR,SAT02_VENOUS,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,TEMPERATURE_MEAN,BLOODPRESSURE_DIASTOLIC_MEDIAN,BLOODPRESSURE_SISTOLIC_MEDIAN,HEART_RATE_MEDIAN,RESPIRATORY_RATE_MEDIAN,TEMPERATURE_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MAX
PATIENT_VISIT_IDENTIFIER,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,0.555556,0,0.0,0.0,1.0,0.0,0.0,1.0,0.109756,1.000000,-0.875236,0.345679,0.086420,-0.230769,-0.283019,-0.285714,0.086420,-0.230769,-0.283019,-0.586207,-0.285714,0.237113,0.0000,-0.162393,-0.500000,-0.420290
2,0,0.000000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.158537,1.000000,-0.939887,0.345679,-0.489712,-0.685470,-0.048218,0.357143,-0.506173,-0.815385,-0.056604,-0.517241,0.357143,-0.525773,-0.5125,-0.111111,-0.714286,0.101449
3,0,0.333333,1,0.0,0.0,0.0,0.0,1.0,1.0,-0.219512,-0.828421,-0.503592,0.580247,0.012346,-0.369231,-0.528302,-0.285714,0.012346,-0.369231,-0.528302,-0.448276,-0.285714,0.175258,-0.1125,-0.384615,-0.357143,-0.420290
4,0,0.000000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.304878,1.000000,-0.990926,0.345679,0.333333,-0.153846,0.160377,0.285714,0.333333,-0.153846,0.160377,-0.586207,0.285714,0.443299,0.0000,0.196581,-0.571429,0.072464
5,0,0.000000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.243902,1.000000,-0.997732,0.345679,-0.037037,-0.538462,-0.537736,-0.196429,-0.037037,-0.538462,-0.537736,-0.517241,-0.196429,0.030928,-0.3750,-0.401709,-0.428571,-0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,0,0.333333,1,0.0,0.0,0.0,0.0,1.0,1.0,-0.195122,1.000000,-0.565974,0.345679,-0.160494,-0.692308,0.339623,0.142857,-0.160494,-0.692308,0.339623,-0.448276,0.142857,0.030928,-0.3750,0.401709,-0.357143,-0.072464
381,1,1.000000,0,0.0,0.0,0.0,0.0,1.0,1.0,-0.292683,1.000000,-0.993195,0.345679,-0.407407,-0.692308,-0.283019,-0.059524,-0.407407,-0.692308,-0.283019,-0.448276,-0.250000,-0.175258,-0.3750,-0.162393,-0.357143,0.072464
382,0,0.444444,0,0.0,0.0,0.0,0.0,0.0,0.0,0.048780,1.000000,-0.034405,0.345679,0.012346,-0.384615,-0.320755,-0.071429,0.012346,-0.384615,-0.320755,-0.448276,-0.071429,0.175258,-0.1250,-0.196581,-0.357143,-0.246377


In [146]:
type(mask)

numpy.ndarray

In [89]:
X_train['AGE_ABOVE65'].to_frame()

,AGE_ABOVE65
PATIENT_VISIT_IDENTIFIER,
190,1
326,1
211,0
170,1
113,1
...,...
322,0
221,0
191,0


In [160]:
X_train, X_test, y_train, y_test = train_test_split(X_1, y, stratify=y, shuffle=True, test_size=0.20)

roc_auc = []
for feature in X_train.columns:
    
    rf_clf = RandomForestClassifier(n_estimators = 100, random_state=102930)
    rf_clf.fit(X_train[feature].to_frame(), y_train)
    y_pred = rf_clf.predict(X_test[feature].to_frame())
    roc_auc.append(roc_auc_score(y_test, y_pred))
    print('Feature:', feature, '; ROC:', roc_auc_score(y_test, y_pred).round(3))

Feature: AGE_ABOVE65 ; ROC: 0.695
Feature: AGE_SCALED ; ROC: 0.579
Feature: GENDER ; ROC: 0.557
Feature: DISEASE GROUPING 1 ; ROC: 0.567
Feature: DISEASE GROUPING 3 ; ROC: 0.489
Feature: DISEASE GROUPING 5 ; ROC: 0.623
Feature: HTN ; ROC: 0.62
Feature: IMMUNOCOMPROMISED ; ROC: 0.567
Feature: OTHER ; ROC: 0.5
Feature: HEMOGLOBIN ; ROC: 0.539
Feature: LACTATE ; ROC: 0.51
Feature: PCR ; ROC: 0.5
Feature: SAT02_VENOUS ; ROC: 0.612
Feature: BLOODPRESSURE_DIASTOLIC_MEAN ; ROC: 0.605
Feature: BLOODPRESSURE_SISTOLIC_MEAN ; ROC: 0.48
Feature: HEART_RATE_MEAN ; ROC: 0.598
Feature: TEMPERATURE_MEAN ; ROC: 0.469
Feature: BLOODPRESSURE_DIASTOLIC_MEDIAN ; ROC: 0.583
Feature: BLOODPRESSURE_SISTOLIC_MEDIAN ; ROC: 0.524
Feature: HEART_RATE_MEDIAN ; ROC: 0.553
Feature: RESPIRATORY_RATE_MEDIAN ; ROC: 0.646
Feature: TEMPERATURE_MEDIAN ; ROC: 0.569
Feature: BLOODPRESSURE_DIASTOLIC_MIN ; ROC: 0.679
Feature: BLOODPRESSURE_SISTOLIC_MIN ; ROC: 0.552
Feature: HEART_RATE_MIN ; ROC: 0.538
Feature: RESPIRATORY_RAT

In [159]:
roc_values = pd.Series(roc_auc)
mask = (roc_values >= 0.52) | (roc_values <= 0.48)
mask = mask.to_numpy()
X_1.loc[:,mask]

,AGE_ABOVE65,AGE_SCALED,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 3,DISEASE GROUPING 5,HTN,HEMOGLOBIN,LACTATE,PCR,SAT02_VENOUS,BLOODPRESSURE_DIASTOLIC_MEAN,TEMPERATURE_MEAN,BLOODPRESSURE_DIASTOLIC_MEDIAN,BLOODPRESSURE_SISTOLIC_MEDIAN,RESPIRATORY_RATE_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MAX
PATIENT_VISIT_IDENTIFIER,,,,,,,,,,,,,,,,,,,,
0,1,0.555556,0,0.0,0.0,1.0,0.0,0.109756,1.000000,-0.875236,0.345679,0.086420,-0.285714,0.086420,-0.230769,-0.586207,0.237113,0.0000,-0.500000,-0.420290
2,0,0.000000,0,0.0,0.0,0.0,0.0,0.158537,1.000000,-0.939887,0.345679,-0.489712,0.357143,-0.506173,-0.815385,-0.517241,-0.525773,-0.5125,-0.714286,0.101449
3,0,0.333333,1,0.0,0.0,0.0,0.0,-0.219512,-0.828421,-0.503592,0.580247,0.012346,-0.285714,0.012346,-0.369231,-0.448276,0.175258,-0.1125,-0.357143,-0.420290
4,0,0.000000,0,0.0,0.0,0.0,0.0,0.304878,1.000000,-0.990926,0.345679,0.333333,0.285714,0.333333,-0.153846,-0.586207,0.443299,0.0000,-0.571429,0.072464
5,0,0.000000,0,0.0,0.0,0.0,0.0,0.243902,1.000000,-0.997732,0.345679,-0.037037,-0.196429,-0.037037,-0.538462,-0.517241,0.030928,-0.3750,-0.428571,-0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,0,0.333333,1,0.0,0.0,0.0,0.0,-0.195122,1.000000,-0.565974,0.345679,-0.160494,0.142857,-0.160494,-0.692308,-0.448276,0.030928,-0.3750,-0.357143,-0.072464
381,1,1.000000,0,0.0,0.0,0.0,0.0,-0.292683,1.000000,-0.993195,0.345679,-0.407407,-0.059524,-0.407407,-0.692308,-0.448276,-0.175258,-0.3750,-0.357143,0.072464
382,0,0.444444,0,0.0,0.0,0.0,0.0,0.048780,1.000000,-0.034405,0.345679,0.012346,-0.071429,0.012346,-0.384615,-0.448276,0.175258,-0.1250,-0.357143,-0.246377


In [93]:
roc_values = pd.Series(roc_auc)
roc_values.index = X_train.columns
roc_values.sort_values(ascending=False, inplace = True)
roc_auc

[0.6259968102073367,
 0.6220095693779905,
 0.5155502392344498,
 0.47767145135566186,
 0.5362838915470494,
 0.5382775119617226,
 0.5442583732057417,
 0.5251196172248804,
 0.5,
 0.5602073365231259,
 0.6271929824561403,
 0.564194577352472,
 0.594896331738437,
 0.5614035087719298,
 0.5291068580542264,
 0.5075757575757576,
 0.45813397129186606,
 0.576555023923445,
 0.4916267942583732,
 0.5318979266347688,
 0.5534290271132376,
 0.4812599681020734,
 0.5837320574162679,
 0.5067783094098883,
 0.5299043062200957,
 0.4856459330143541,
 0.48245614035087714]

<a name="clean"></a>
# Salva dados 

O DataFrame: ```df_7_cleaned``` será salvo como csv na pasta ```../data/interim/```. Isso porque os dados podem ainda sofrer alterações na etapa de análise das *features*.

In [35]:
df_2_featured.to_csv('../data/interim/df_featured.csv')

-----------------
Retornar ao [Sumário](#sumario)

In [36]:
# Compute the correlation matrix
corr = df_1_agescaled.iloc[:,12:-2].corr().abs()
masked_corr = corr
masked_corr[corr <= 0.95] = 0
masked_corr[corr > 0.95] = 1

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(15, 15))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
#sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,square=True, linewidths=.5, cbar_kws={"shrink": .5})
sns.heatmap(masked_corr, mask=mask, cmap=cmap, center=0, square=True, vmin=0, vmax=1.0, cbar=False)

NameError: name 'plt' is not defined

In [35]:
upper_tri = masked_corr.where(np.triu(np.ones(masked_corr.shape),k=1).astype(np.bool))
correlated = []
for i in range(72):
    for j in range(72):
        if upper_tri.iloc[i,j] > 0.95:
            print(f'{upper_tri.index[i]} x {upper_tri.columns[j]}')
            correlated.append([upper_tri.index[i], upper_tri.columns[j]])

HEMATOCRITE x HEMOGLOBIN
LEUKOCYTES x NEUTROPHILES
BLOODPRESSURE_DIASTOLIC_MEAN x BLOODPRESSURE_DIASTOLIC_MEDIAN
BLOODPRESSURE_DIASTOLIC_MEAN x BLOODPRESSURE_DIASTOLIC_MIN
BLOODPRESSURE_SISTOLIC_MEAN x BLOODPRESSURE_SISTOLIC_MEDIAN
BLOODPRESSURE_SISTOLIC_MEAN x BLOODPRESSURE_SISTOLIC_MIN
BLOODPRESSURE_SISTOLIC_MEAN x BLOODPRESSURE_SISTOLIC_MAX
HEART_RATE_MEAN x HEART_RATE_MEDIAN
HEART_RATE_MEAN x HEART_RATE_MIN
HEART_RATE_MEAN x HEART_RATE_MAX
RESPIRATORY_RATE_MEAN x RESPIRATORY_RATE_MEDIAN
RESPIRATORY_RATE_MEAN x RESPIRATORY_RATE_MAX
TEMPERATURE_MEAN x TEMPERATURE_MEDIAN
TEMPERATURE_MEAN x TEMPERATURE_MIN
TEMPERATURE_MEAN x TEMPERATURE_MAX
OXYGEN_SATURATION_MEAN x OXYGEN_SATURATION_MEDIAN
OXYGEN_SATURATION_MEAN x OXYGEN_SATURATION_MIN
OXYGEN_SATURATION_MEAN x OXYGEN_SATURATION_MAX
BLOODPRESSURE_DIASTOLIC_MEDIAN x BLOODPRESSURE_DIASTOLIC_MIN
BLOODPRESSURE_SISTOLIC_MEDIAN x BLOODPRESSURE_SISTOLIC_MIN
BLOODPRESSURE_SISTOLIC_MEDIAN x BLOODPRESSURE_SISTOLIC_MAX
HEART_RATE_MEDIAN x HEART_RA

<ipython-input-35-51e76a09787c>:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper_tri = masked_corr.where(np.triu(np.ones(masked_corr.shape),k=1).astype(np.bool))
